In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

c:\Users\User\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
c:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [17]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [94]:
df = pd.read_parquet("./fhv_tripdata_2021-01.parquet")
df.head()
df.info()

len(df.index)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1154112 entries, 0 to 1154111
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1154112 non-null  object        
 1   pickup_datetime         1154112 non-null  datetime64[ns]
 2   dropOff_datetime        1154112 non-null  datetime64[ns]
 3   PUlocationID            195845 non-null   float64       
 4   DOlocationID            991892 non-null   float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1153227 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 61.6+ MB


1154112

In [95]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df = df[(df.duration >= 1) & (df.duration <= 60)]
categorical = ['PUlocationID', 'DOlocationID']
df[categorical] = df[categorical].astype(str)
df['PUlocationID'] = df['PUlocationID'].replace('nan', "-1")
df['DOlocationID'] = df['DOlocationID'].replace('nan', "-1")

In [96]:
df.describe()

,duration
count,1.109826e+06
mean,1.624725e+01
std,1.155150e+01
min,1.000000e+00
25%,7.850000e+00
50%,1.323333e+01
75%,2.146667e+01
max,6.000000e+01


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109826 entries, 0 to 1154111
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1109826 non-null  object        
 1   pickup_datetime         1109826 non-null  datetime64[ns]
 2   dropOff_datetime        1109826 non-null  datetime64[ns]
 3   PUlocationID            1109826 non-null  object        
 4   DOlocationID            1109826 non-null  object        
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1109053 non-null  object        
 7   duration                1109826 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 76.2+ MB


In [98]:
df['duration'].nunique()

3541

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109826 entries, 0 to 1154111
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1109826 non-null  object        
 1   pickup_datetime         1109826 non-null  datetime64[ns]
 2   dropOff_datetime        1109826 non-null  datetime64[ns]
 3   PUlocationID            1109826 non-null  object        
 4   DOlocationID            1109826 non-null  object        
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1109053 non-null  object        
 7   duration                1109826 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(5)
memory usage: 76.2+ MB


In [100]:
(1109826 - 182818 )/ 1109826 

0.8352732770722617

In [101]:
df.shape

(1109826, 8)

In [102]:
df = df.fillna(-1)
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1,-1,-1,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1,-1,-1,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1,72.0,-1,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1,61.0,-1,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1,71.0,-1,B00037,9.050000


In [103]:
categorical = ['PUlocationID', 'DOlocationID'] #'PULocationID', 'DOLocationID']
#numerical = ['duration']

dv = DictVectorizer()

train_dicts = df[categorical ].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# val_dicts = df[categorical + numerical].to_dict(orient='records')
# X_val = dv.transform(val_dicts)

In [104]:
y_train = df['duration']

In [105]:
X_train.get_shape()

(1109826, 525)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037


## DF val, process and transformation

In [106]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
    df.pickup_datetime = pd.to_datetime(df.pickup_datetime)

    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

df_val = read_dataframe("./fhv_tripdata_2021-02.parquet")
df_val = df_val.fillna(-1)


In [107]:
df_val['PUlocationID'] = df_val['PUlocationID'].replace('nan', "-1")
df_val['DOlocationID'] = df_val['DOlocationID'].replace('nan', "-1")

In [108]:
categorical =['PUlocationID', 'DOlocationID'] #'PULocationID', 'DOLocationID']


val_dicts = df_val[categorical ].to_dict(orient='records')
X_val = dv.fit_transform(val_dicts)

In [109]:
y_val = df_val['duration']

In [110]:
X_val.get_shape()

(990113, 526)

In [111]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990113 entries, 1 to 1037691
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   dispatching_base_num    990113 non-null  object        
 1   pickup_datetime         990113 non-null  datetime64[ns]
 2   dropOff_datetime        990113 non-null  datetime64[ns]
 3   PUlocationID            990113 non-null  object        
 4   DOlocationID            990113 non-null  object        
 5   SR_Flag                 990113 non-null  int64         
 6   Affiliated_base_number  990113 non-null  object        
 7   duration                990113 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(1), object(4)
memory usage: 68.0+ MB


In [115]:
df['DOlocationID'].unique()

array(['-1', '72.0', '61.0', '71.0', '91.0', '39.0', '37.0', '89.0',
       '177.0', '225.0', '63.0', '67.0', '22.0', '14.0', '188.0', '35.0',
       '122.0', '168.0', '77.0', '237.0', '196.0', '60.0', '126.0',
       '119.0', '169.0', '254.0', '213.0', '174.0', '248.0', '259.0',
       '42.0', '32.0', '18.0', '127.0', '69.0', '121.0', '240.0', '20.0',
       '242.0', '200.0', '182.0', '208.0', '70.0', '129.0', '16.0',
       '252.0', '223.0', '78.0', '136.0', '167.0', '173.0', '82.0',
       '142.0', '159.0', '244.0', '147.0', '116.0', '206.0', '115.0',
       '245.0', '221.0', '187.0', '76.0', '258.0', '210.0', '17.0',
       '189.0', '181.0', '228.0', '222.0', '247.0', '85.0', '198.0',
       '256.0', '10.0', '66.0', '49.0', '232.0', '108.0', '152.0',
       '220.0', '153.0', '3.0', '241.0', '265.0', '106.0', '235.0',
       '141.0', '113.0', '132.0', '134.0', '197.0', '28.0', '102.0',
       '130.0', '94.0', '243.0', '47.0', '107.0', '112.0', '40.0', '21.0',
       '75.0', '212.0',

## Train the model and calculate RMSE 

In [117]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [116]:

y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

10.528519107205634